<a href="https://colab.research.google.com/github/federicotarozzi/Corso_AnalistaProgrammatoreDatiGeoSpaziali/blob/main/4_NDVI_timeseries_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Average field's NDVI estraction notebook
from the NDVI satellite images, the average NDVI for each tomato-growing field will be extracted with this notebook. To do this, image processing is required to remove data influenced by the presence of clouds. Second, each polygon will be cropped to the image and the average NDVI for each date will be extracted.

- shapefile (to transform with qGIS): https://agreagestione.regione.emilia-romagna.it/agrea-file/PianiColturaliAlfanumerici/
- NDVI images: EO:CLMS:DAT:CLMS_GLOBAL_NDVI_300M_V1_10DAILY_NETCDF at https://www.wekeo.eu/data?view=viewer (Wekeo platform) from 1/04 - 30/09 : the growing period

#### Import the library we need

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'

import rasterio
import fiona
from shapely.geometry import shape, mapping, Polygon
from shapely.wkt import loads
from shapely import wkt
from fiona.transform import transform_geom
from rasterio.mask import mask
import numpy as np
import numpy.ma as ma
import glob

import pandas as pd


# Disable the warning for invalid values encountered in casting
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

#### Define the data paths

In [ ]:
img_folder = "Data/NDVI_data/piacenza/2019/NDVI" # NDVI images folder
qf_folder = "Data/NDVI_data/piacenza/2019/QF" # Quality flag images folder - cloud detection
output_folder = "Data/NDVI_data/piacenza/2019/NDVI_corrected" #where to put the pocessed images
shp = "Data/NDVI_data/piacenza/2019/shapefile_input/pomodoro_campi50_2019.shp" # shapefile with choosen fields

In [ ]:
#select all the tiff images in the folders
img_paths = glob.glob(os.path.join(img_folder, '*.tif'))
qf_paths = glob.glob(os.path.join(qf_folder, '*.tif'))

In [ ]:
#define a funtion to find the geometry and save in WKT format (Well-Known Text)
def reshape(geom):
    return shape(geom).buffer(0).wkt


# Read the shapefile
with fiona.open(shp, "r") as shapefile:
    shapefile_crs = shapefile.crs #defining the actual crs
    shapes = [feature["geometry"] for feature in shapefile] # geometry list

    # Transform geometries to EPSG:32632
    # if I dont want the reshape
    transformed_shapes = [transform_geom(shapefile_crs, "EPSG:32632", shape) for shape in shapes]

    # For the creation of the dataframe extract properties and transformed geometries and reshape them in a readable format
    df_transformed_shapes = [reshape(transform_geom(shapefile_crs, "EPSG:32632", feature["geometry"])) for feature in shapefile]
    data = []
    for feature, s in zip(shapefile, df_transformed_shapes):
        properties = feature['properties']
        data.append({'id': properties['id'], 'geometry': s, 'area': properties.get('area', None)})

# Create a DataFrame
polygon_df = pd.DataFrame(data)
polygon_df['geometry'] = polygon_df['geometry'].astype(str)

# Display the DataFrame
print(polygon_df.head())

CRSError: The WKT could not be parsed. PROJ: internal_proj_create_from_database: /opt/conda/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

#### Mask all the NDVI rasters with the Quality flag rasters

In [ ]:
#find the date of every images based on images names with lambda function
get_date = lambda path: os.path.basename(path).split('_')[1]

In [ ]:
# Sort the image and qf files based on the parsed date
sorted_img_paths = sorted(img_paths, key=lambda path: get_date(path))
sorted_qf_paths = sorted(qf_paths, key=lambda path: get_date(path))

# Print the sorted paths
for img_path, qf_path in zip(sorted_img_paths, sorted_qf_paths):
    # Read the image and quality files
    with rasterio.open(img_path) as src_img, rasterio.open(qf_path) as src_qf:
        #print("Raster CRS:", src_img.crs)
        profile = src_img.profile
        nodata_value = profile.get('nodata')

        # Mask the image and quality files with the shapefile
        img, img_transform = mask(src_img, transformed_shapes, crop=True, filled=True, nodata=nodata_value)
        qf, qf_transform = mask(src_qf, transformed_shapes, crop=True, filled=True)

        # Replace 0 values with NaN outside transformed_shapes
        img = np.where(img == 0, np.nan, img)


        # New image only where qf = 1 while preserving nan values of the original image
        #clean_img = np.where((qf == 1) | np.isnan(img), img, np.nan)
        clean_img = np.where(qf == np.uint16(1), img, np.nan)
        clean_img[np.isnan(clean_img)] = nodata_value

        # Update the metadata for the masked image
        img_meta = src_img.meta.copy()
        img_meta.update({"driver": "GTiff",
                         "height": img.shape[1],
                         "width": img.shape[2],
                         "transform": img_transform,
                         "nodata": nodata_value
                         }
                         )

        # Save the masked image into a new folder
        output_img_path = os.path.join(output_folder, f"masked_{os.path.basename(img_path)}")

        #print(img.dtype)
        with rasterio.open(output_img_path, "w", **img_meta) as dest_img:
            dest_img.write(clean_img[0, :, :].astype("int16"), 1)

In [ ]:
# Sort the image and qf files based on the parsed date
sorted_img_paths = sorted(img_paths, key=lambda path: get_date(path))
sorted_qf_paths = sorted(qf_paths, key=lambda path: get_date(path))

# Print the sorted paths
for img_path, qf_path in zip(sorted_img_paths, sorted_qf_paths):
    # Read the image and quality files
    with rasterio.open(img_path) as src_img, rasterio.open(qf_path) as src_qf:
        #print("Raster CRS:", src_img.crs)
        profile = src_img.profile
        nodata_value = profile.get('nodata')

        # Mask the image and quality files with the shapefile
        img, img_transform = mask(src_img, transformed_shapes, crop=True, filled=True, nodata=nodata_value)
        qf, qf_transform = mask(src_qf, transformed_shapes, crop=True, filled=True)

        # Replace 0 values with NaN outside transformed_shapes
        img = np.where(img == 0, np.nan, img)


        # New image only where qf = 1 while preserving nan values of the original image
        #clean_img = np.where((qf == 1) | np.isnan(img), img, np.nan)
        clean_img = np.where(qf == np.uint16(1), img, np.nan)
        clean_img[np.isnan(clean_img)] = nodata_value

        # Update the metadata for the masked image
        img_meta = src_img.meta.copy()
        img_meta.update({"driver": "GTiff",
                         "height": img.shape[1],
                         "width": img.shape[2],
                         "transform": img_transform,
                         "nodata": nodata_value
                         }
                         )

        # Save the masked image into a new folder
        output_img_path = os.path.join(output_folder, f"masked_{os.path.basename(img_path)}")

        #print(img.dtype)
        with rasterio.open(output_img_path, "w", **img_meta) as dest_img:
            dest_img.write(clean_img[0, :, :].astype("int16"), 1)

_________________________________________________________________________________________________________________________________

Calcolo della media e mediana e creazione del dataframe

In [ ]:
def compute_statistics(src, polygon_geometry, threshold):
    polygon = shape(polygon_geometry)

    # Create a GeoJSON-like object with CRS
    polygon_geojson = mapping(polygon)

    # Mask the image using the GeoJSON-like object
    masked_image, _ = mask(src, [polygon_geojson], crop=True)

    # Since i'm working with one polygon at a time, i can check here if the covered area is > than a certain threshold
    covered_area_pixels = np.nansum(masked_image)
    # converts the polygon's geometric area to an area in terms of pixels.
    total_polygon_area_pixels = polygon.area / src.res[0] * src.res[1]
    covered_percentage = (covered_area_pixels / total_polygon_area_pixels) * 100

    if covered_percentage > threshold:
        # if the covered area is > than the threshold then it doesnt make sense compute the mean and median
        mean_value = np.nan
        median_value = np.nan
    else:
        # Compute the mean and median pixel value
        mean_value = np.nanmean(masked_image)
        median_value = np.nanmedian(masked_image)


    return mean_value, median_value

In [ ]:
def get_date(image_path):
    date_str = os.path.basename(image_path).split("_")[3][:8]
    date_obj = pd.to_datetime(date_str, format='%Y%m%d')
    return date_obj.date()

In [ ]:
image_paths = sorted([os.path.join(output_folder, img) for img in os.listdir(output_folder) if img.endswith(".tif")], key=lambda path: get_date(path))

In [ ]:
header = ["Polygon_ID", "Coordinates", "Date", "Mean_NDVI_per_pixel", "Median_NDVI_per_pixel"]
rows = []

polygon_df = polygon_df.sort_values(by='id')
sorted_image_paths = sorted(image_paths, key=lambda path: get_date(path))

for image_name in sorted_image_paths:
    if image_name.endswith(".tif"):
        image_path = os.path.abspath(image_name)
        date = get_date(image_path)

        # Read the image data once
        with rasterio.open(image_path) as src:
            # Iterate through each polygon and calculate mean pixel value
            # edit: only if i have a non covered area > threshold
            for _, row in polygon_df.iterrows():
                poly_id = row['id']
                polygon = row['geometry']
                area = row['area']

                polygon = loads(polygon)

                # Calculate mean pixel value
                mean_value, median_value = compute_statistics(src, polygon, 0.8)

                # Append data as a list
                row_data = [poly_id, polygon, date, mean_value, median_value]  # Replace 0 with the actual mean pixel value
                rows.append(row_data)

# Create DataFrame from the list of lists
result_df = pd.DataFrame(rows, columns=header)
result_df = result_df.sort_values(by=['Polygon_ID', 'Date'])

In [ ]:
result_df

______________________________________________________________________________________________________________________

#### Save dataset in CSV

In [ ]:
result_df.to_csv('Data/NDVI_data/NDVI_50_pc_2019.csv', index=False)